In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [3]:
"""
Load dataset.
"""
dataset = load_dataset("yelp_review_full")
print(dataset)

Generating test split: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:00<00:00, 282191.98 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [5]:
"""
Tokenize dataset.
"""
# init tokenizer
model_id = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize(X):
    return tokenizer(
        X["text"],
        padding="max_length",
        truncation=True,
    )

# tokenize data
tokenized_dataset = dataset.map(tokenize, batched=True)

C:\Users\Admin\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--google-bert--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████